# youtube download

In [ ]:
# # pip install yt-dlp spotipy-
# brew install ffmpeg  # macOS
# sudo apt install ffmpeg  # Ubuntu/Debian

In [1]:
import yt_dlp

def download_playlist(playlist_url, mode='mp3'):
    ydl_opts = {}

    if mode == 'mp3':
       ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': '%(title)s.%(ext)s',
    'ignoreerrors': True,  # <- this line skips private/unavailable videos
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',
    }],
}

    elif mode == '720p':
        ydl_opts = {
            'format': 'bestvideo[height<=720]+bestaudio/best[height<=720]',
            'outtmpl': '%(title)s.%(ext)s',
            'merge_output_format': 'mp4',
        }

    elif mode == '1080p':
        ydl_opts = {
            'format': 'bestvideo[height<=1080]+bestaudio/best[height<=1080]',
            'outtmpl': '%(title)s.%(ext)s',
            'merge_output_format': 'mp4',
        }

    else:
        print("Invalid mode. Choose from: mp3, 720p, 1080p")
        return

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([playlist_url])

# === Example usage ===
playlist_link = input("Enter YouTube playlist URL: ")
mode = input("Enter format (mp3 / 720p / 1080p): ").lower()
download_playlist(playlist_link, mode)

[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PL4Z9tfKYkdBMI14jR-4Gh-yFOmXwGQx13&si=9MQu4h1ls2-GAQ91
[youtube:tab] PL4Z9tfKYkdBMI14jR-4Gh-yFOmXwGQx13: Downloading webpage
[youtube:tab] PL4Z9tfKYkdBMI14jR-4Gh-yFOmXwGQx13: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: video
[youtube:tab] PL4Z9tfKYkdBMI14jR-4Gh-yFOmXwGQx13 page 1: Downloading API JSON
[youtube:tab] Playlist video: Downloading 1 items of 1
[download] Downloading item 1 of 1
[youtube] Extracting URL: https://www.youtube.com/watch?v=Y7jDh7aOQCY
[youtube] Y7jDh7aOQCY: Downloading webpage
[youtube] Y7jDh7aOQCY: Downloading tv client config
[youtube] Y7jDh7aOQCY: Downloading tv player API JSON
[youtube] Y7jDh7aOQCY: Downloading ios player API JSON
[youtube] Y7jDh7aOQCY: Downloading m3u8 information
[info] Y7jDh7aOQCY: Downloading 1 format(s): 335+251
[download] Destination: Underwater Life OLED Demo HDR 8K Ultra HD Dolby Vision™ 60fps.f335.webm
[download] 100% of  998

# spotify downloder

In [ ]:
import yt_dlp
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def get_spotify_tracks(playlist_url, client_id, client_secret):
    """Fetch track names from Spotify playlist."""
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    ))

    # Extract playlist ID
    playlist_id = playlist_url.split("/")[-1].split("?")[0]

    tracks = []
    results = sp.playlist_tracks(playlist_id)

    while results:
        for item in results['items']:
            track = item['track']
            if track:
                artist = track['artists'][0]['name']
                title = track['name']
                search_query = f"{artist} - {title}"
                tracks.append(search_query)

        if results['next']:
            results = sp.next(results)
        else:
            break

    return tracks

def download_tracks_from_youtube(track_list):
    """Search and download tracks from YouTube as MP3 (320kbps)."""
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'ignoreerrors': True,
        'quiet': False,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        for track in track_list:
            print(f"\n🔍 Searching & downloading: {track}")
            try:
                ydl.download([f"ytsearch1:{track}"])
            except Exception as e:
                print(f"❌ Failed to download {track}: {e}")

def main():
    print("🎧 Spotify Playlist to MP3 Downloader\n")

    playlist_url = input("🔗 Enter Spotify playlist URL: ").strip()
    client_id = input("🔐 Enter your Spotify CLIENT ID: ").strip()
    client_secret = input("🔐 Enter your Spotify CLIENT SECRET: ").strip()

    print("\n📥 Fetching tracks from Spotify playlist...")
    track_list = get_spotify_tracks(playlist_url, client_id, client_secret)

    print(f"🎵 Found {len(track_list)} tracks. Starting download...\n")
    download_tracks_from_youtube(track_list)

    print("\n✅ Done!")

if __name__ == '__main__':
    main()

youtube according to avaibility


In [ ]:
import yt_dlp

def list_formats(video_url):
    print("\n🔍 Fetching available formats...\n")
    ydl_opts = {
        'quiet': True,
        'no_warnings': True,
        'skip_download': True,
        'forcejson': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(video_url, download=False)
        formats = info_dict.get('formats', [])

        resolution_map = {}
        for f in formats:
            height = f.get('height')
            if height and f.get('vcodec') != 'none' and f.get('acodec') == 'none':  # video-only formats
                label = f"{height}p"
                resolution_map[label] = f['format_id']

        print("🧾 Available resolutions:")
        for res in sorted(resolution_map.keys(), key=lambda x: int(x.replace('p', ''))):
            print(f"  - {res}")

        return resolution_map

def download_video(video_url, resolution):
    # Check available formats
    formats = list_formats(video_url)

    if resolution not in formats:
        print(f"❌ {resolution} is not available for this video.")
        return

    format_id = formats[resolution]
    print(f"\n📥 Downloading in {resolution}...")

    ydl_opts = {
        'format': f'{format_id}+bestaudio',
        'outtmpl': '%(title)s_%(resolution)s.%(ext)s',
        'merge_output_format': 'mp4',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])


video_link = input("🎬 Enter YouTube video URL: ").strip()
available_resolutions = list_formats(video_link)

chosen_res = input("\n🎯 Choose resolution in format as available ").strip()
download_video(video_link, chosen_res)


🔍 Fetching available formats...

🧾 Available resolutions:
  - 144p
  - 240p
  - 360p
  - 480p
  - 720p
  - 1080p

🔍 Fetching available formats...

🧾 Available resolutions:
  - 144p
  - 240p
  - 360p
  - 480p
  - 720p
  - 1080p

📥 Downloading in 1080p...
[youtube] Extracting URL: https://youtu.be/FLbE0IkHLkg?si=iNsF2HXJLC1JlIAi
[youtube] FLbE0IkHLkg: Downloading webpage
[youtube] FLbE0IkHLkg: Downloading tv client config
[youtube] FLbE0IkHLkg: Downloading tv player API JSON
[youtube] FLbE0IkHLkg: Downloading ios player API JSON
[youtube] FLbE0IkHLkg: Downloading m3u8 information
[info] Testing format 616
[info] FLbE0IkHLkg: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 49
[download] Destination: Furfuri - Kuma Sagar_1920x1080.f616.mp4
[download] 100% of   13.57MiB in 00:00:02 at 6.24MiB/s                   
[download] Destination: Furfuri - Kuma Sagar_1920x1080.f251.webm
[download] 100% of    3.95MiB in 00:00:00 at 5.47MiB/s   
[Mer